In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
title = 'Analytics Engineer'
location = 'San Francisco' 

In [ ]:
# {Locations:'San Francisco','San Jose','New York','Seattle'}
# {jobs: 'Data Analyst','Analytics Engineer','Data Scientist','Product Manager'}

# For the above, the same script used in this file can be run except for replacing the URL in the below code block
# Filtering each of the above jobs for each of the above locations gives different URL which can be replaced
# It will run and provide the respective jobs in the respective locations and dump it in separate CSV file

In [4]:
job_list = []

for page_start in range(0, 1000, 100):  
    url = f'https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=Analytics%2BEngineers&location=San%20Francisco%2C%20CA&start={page_start}'
    response = requests.get(url)
    list_soup = BeautifulSoup(response.text, "html.parser")
    page_jobs = list_soup.find_all("li")

    for job in page_jobs:
        base_card_div = job.find("div", {"class": "base-card"})
        job_id = base_card_div.get("data-entity-urn").split(":")[3]

        job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
        job_response = requests.get(job_url)
        job_soup = BeautifulSoup(job_response.text, 'html.parser')
        job_post = {}

        try:
            job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link topcard__flavor--black-link"}).text.strip()
        except:
            job_post["company_name"] = None
        try:
            job_post["job_title"] = job_soup.find("h2", {"class": "top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title"}).text.strip()
        except:
            job_post["job_title"] = None
        try:
            job_post["location"] = job_soup.find("span", {"class": "topcard__flavor topcard__flavor--bullet"}).text.strip()
        except:
            job_post["location"] = None
        try:
            job_post["num_applicants"] = job_soup.find("span", {"class": "posted-time-ago__text topcard__flavor--metadata"}).text.strip()
        except:
            job_post["num_applicants"] = None

        job_list.append(job_post)


In [5]:
jobs_df = pd.DataFrame(job_list)
print(jobs_df)

                           company_name  \
0                                 Chime   
1                        FocusKPI, Inc.   
2             San Francisco Health Plan   
3                                  Meta   
4                              Airtable   
..                                  ...   
95                         IXL Learning   
96                                 Cube   
97              Recruiting from Scratch   
98              Recruiting from Scratch   
99  myGwork - LGBTQ+ Business Community   

                                            job_title           location  \
0                      Business Intelligence Engineer  San Francisco, CA   
1                                 Junior Data Analyst  San Francisco, CA   
2                               Insights Data Analyst  San Francisco, CA   
3                            Data Engineer, Analytics     Burlingame, CA   
4                                  Analytics Engineer  San Francisco, CA   
..                         

In [6]:
jobs_df = pd.DataFrame(job_list)
jobs_df.to_csv('Analytics_Engineer_SF.csv', index=False)